<a href="https://colab.research.google.com/github/Syilun/AgeGenderDeepLearning/blob/master/AgeGenderDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Age and Gender Classification Using Convolutional Neural Networks - Demo

This code is released with the paper:

Gil Levi and Tal Hassner, "Age and Gender Classification Using Convolutional Neural Networks," IEEE Workshop on Analysis and Modeling of Faces and Gestures (AMFG), at the IEEE Conf. on Computer Vision and Pattern Recognition (CVPR), Boston, June 2015

If you find the code useful, please add suitable reference to the paper in your work.

In [1]:
!git clone https://github.com/BVLC/caffe.git

fatal: destination path 'caffe' already exists and is not an empty directory.


In [2]:
!sudo apt install libboost1.58-all-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libatlas-base-dev libgflags-dev libgoogle-glog-dev liblmdb-dev python-pip python-scipy python3-scipy -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libboost1.58-all-dev
E: Couldn't find any package by glob 'libboost1.58-all-dev'
E: Couldn't find any package by regex 'libboost1.58-all-dev'


In [3]:
!apt install -y caffe-cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
caffe-cuda is already the newest version (1.0.0-6build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [4]:
!git clone https://github.com/GilLevi/AgeGenderDeepLearning.git

fatal: destination path 'AgeGenderDeepLearning' already exists and is not an empty directory.


In [5]:
!cp /content/AgeGenderDeepLearning/models/mean.binaryproto /content/caffe/mean.binaryproto

!cp /content/AgeGenderDeepLearning/models/age_net.caffemodel /content/caffe/age_net.caffemodel
!cp /content/AgeGenderDeepLearning/age_net_definitions/deploy.prototxt /content/caffe/deploy_age.prototxt

!cp /content/AgeGenderDeepLearning/models/gender_net.caffemodel /content/caffe/gender_net.caffemodel
!cp /content/AgeGenderDeepLearning/gender_net_definitions/deploy.prototxt /content/caffe/deploy_gender.prototx

In [13]:
import caffe
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


caffe_root = '/content/caffe/' 



plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

ImportError: ignored

## Loading the mean image

In [ ]:
mean_filename='/content/caffe/mean.binaryproto'
proto_data = open(mean_filename, "rb").read()
a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data)
mean  = caffe.io.blobproto_to_array(a)[0]

## Loading the age network

In [ ]:
age_net_pretrained='./age_net.caffemodel'
age_net_model_file='./deploy_age.prototxt'
age_net = caffe.Classifier(age_net_model_file, age_net_pretrained,
                       mean=mean,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

## Loading the gender network

In [ ]:
gender_net_pretrained='./gender_net.caffemodel'
gender_net_model_file='./deploy_gender.prototxt'
gender_net = caffe.Classifier(gender_net_model_file, gender_net_pretrained,
                       mean=mean,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

## Labels

In [ ]:
age_list=['(0, 2)','(4, 6)','(8, 12)','(15, 20)','(25, 32)','(38, 43)','(48, 53)','(60, 100)']
gender_list=['Male','Female']

## Reading and plotting the input image

In [ ]:
example_image = './example_image.jpg'
input_image = caffe.io.load_image(example_image)
_ = plt.imshow(input_image)

## Age prediction

In [ ]:
prediction = age_net.predict([input_image]) 

print 'predicted age:', age_list[prediction[0].argmax()]

## Gender prediction

In [ ]:
prediction = gender_net.predict([input_image]) 

print 'predicted gender:', gender_list[prediction[0].argmax()]

# Filters visualizations

Based on a similar notebook by Caffe's authors.

In [ ]:
def showimage(im):
    if im.ndim == 3:
        im = im[:, :, ::-1]
    plt.set_cmap('jet')
    plt.imshow(im,vmin=0, vmax=0.3)
    

def vis_square(data, padsize=1, padval=0):
    data -= data.min()
    data /= data.max()
    
    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))
    
    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    
    showimage(data)

### Input image

In [ ]:
_ = plt.imshow(input_image)

###The first conv layer filters, `conv1`
Notice some filters resemble edge detector filters

In [ ]:
filters = age_net.params['conv1'][0].data[:49]
vis_square(filters.transpose(0, 2, 3, 1))

### The first Conv layer output, `conv1` (rectified responses of the filters above)
Notice the response of specific filters, e.g. the onese that resemble edge detectors 

In [ ]:
feat = age_net.blobs['conv1'].data[4, :49]
vis_square(feat, padval=1)

Copyright 2015, Gil Levi and Tal Hassner

The SOFTWARE provided in this page is provided "as is", without any guarantee made as to its suitability or fitness for any particular use. It may contain bugs, so use of this tool is at your own risk. We take no responsibility for any damage of any sort that may unintentionally be caused through its use.